## Bayes module

### Optimisation of Catoni's bound

Given a score $S$, a parametric family of distributions $(\nu_{\theta})_{\theta \in \Theta}$ and a prior distribution $\pi$, we consider the minimisation problem

$$\hat{\theta} = \arg\inf \nu_{\theta}[S] + \lambda * KL(\nu_{\theta}, \pi).$$

The function pacbayes_minimize is designed to tackle such problems in the setting where $\pi =  \nu_{\theta_0}$. This is in order to benefit from potential closed form expressions when computing the Kullback-Leibler divergence and its derivative.

# SurPAC-CE algorithm

SurPAC-CE (Surrogate PAC-Bayes for Catoni's bound minimisation on Exponential family) algorithm relies on surpbayes.proba submodule. The current form of the algorithm requires the family of distributions of interest to be an exponential family (classes "surpbayes.proba.PreExpFamily" and "surpbayes.proba.ExponentialFamily"). A slightly modified version is used in the case of Gaussian distributions (inherited from "surpbayes.proba.PreExpFamily").

To demonstrate SurPAC-CE, a one dimensional problem is considered. However, SurPAC-CE can be used for problems of any dimension, although for large dimensions (> 100), some inner mechanisms (the weighing process) should be assessed and potentially modified. First, we define a tailor-made error function. For maximum efficiency, this function is vectorized.

Note that for demonstration reasons, the hyperparameters of the algorithm are chosen as really conservative. This implies that the algorithm takes more time to converge than it could, and evaluates the score many more time than is really necessary. The resulting graphs are much smoother and the training mechanism better explained.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from surpbayes.bayes import pacbayes_minimize

from surpbayes.proba import GaussianMap, TensorizedGaussianMap, BlockDiagGaussMap

xs = np.linspace(-10, 10, 10**3)

def fun(x):
    return (np.tan(np.sqrt(1 + np.arctan(x - 1.57) ** 2) - 1.0) - 3) ** 2


def gun(x):
    return (np.tanh(x) + 1) / 2


def S(x):
    return (2 - fun(0.7 * (x - 0.4))) * gun(-0.2 - 0.7 * (x - 0.4)) + 3


def score(x):
    return (S(x[:, 0]+2) +.25) /3.25

plt.plot(xs, score(xs[:, np.newaxis]), label="Score fun.")
plt.xlabel("param")
plt.ylabel("Score")
plt.legend()
plt.show()

Calls to SurPAC-CE can be performed using the pacbayes_minimize function from bayes module as follow

In [ ]:
from surpbayes.bayes import pacbayes_minimize
from surpbayes.proba import GaussianMap

from time import time

gmap = GaussianMap(1)

tic = time()
opt_res_new = pacbayes_minimize(
    score,
    gmap,
    # As not specified, prior used is the reference of gmap (i.e. standard Gaussian)
    # As not specified, initial guess for posterior is the prior 
    optimizer="SurPAC-CE",
    temperature=0.015,  
    # Hyperparameters
    per_step=160, # score evaluations per step
    chain_length=51,
    n_estim_weights=10**4,
    # regularisation parameters
    dampen=0.7,
    kl_max=0.1,
    # dichotomy hyperparams for regularisation (can be omitted)
    m_max=25,
    xtol=10**-4,
    kltol=10**-4,
    # function call information
    parallel=False,
    vectorized=True,
    # printing params
    print_rec=5,
)
tac = time()
print("Time elapsed:", tac - tic)

The algorithm converged in less than 20 steps.
The evolution of the approximation of the posterior can be easily represented in this 1D setting by plotting the densities.

In [ ]:
xs = np.linspace(-5.0, 3.4, 2000)


def repr_gauss(param):
    distr = gmap(param)
    return xs, distr.dens(xs[:, np.newaxis])


plt.plot(*repr_gauss(opt_res_new.hist_param[0]), linewidth=1.0, label="prior")
for i, param in enumerate(opt_res_new.hist_param[:]):
    if i % 1 == 0:

        xs, ys = repr_gauss(param)
        plt.plot(xs, ys, color="black", linewidth=0.2)

plt.plot(*repr_gauss(opt_res_new.hist_param[-1]), linewidth=1.0, label="posterior")
plt.title("Evolution of the posterior estimation")
plt.legend()
plt.show()

Let us see what happens in detail during this training phase.

In [ ]:
from surpbayes.bayes.surpac.weighing import get_weights_mc
from surpbayes.bayes.surpac.surpac_solver import exp_approximation
from scipy.stats import norm

np.random.seed(0)

temp = 0.02
dampen = 0.975
n_sample = 40

xs = np.linspace(-3.5, 3.5, 1000)

prior_param = gmap.ref_param
prior = gmap(prior_param)
prior_norm = norm(loc=prior.means[0], scale=np.sqrt(prior.cov[0, 0]))

sample = prior(n_sample)
score_sample = score(sample)

# Plotting
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax2.set_ylabel("Density")
ax1.set_ylabel("Score")

ax2.fill_between(xs, prior_norm.pdf(xs), color="0.7", alpha=0.2, label="prior")
ax1.plot(
    xs, score(xs[:, np.newaxis]), "--", linewidth=1.0, color="black", label="Score"
)

ax1.plot(sample, score_sample, "o", markersize=6.0, label="Score eval.")
ax1.set_ylim(-0.3, 3.2)
ax1.legend(loc=1)
ax2.legend(loc=2)

fig.tight_layout()
plt.show()

Parameters are drawn from the current posterior approximation. The scores of these parameters are evaluated and added to the stack of parameters evaluated so far.

In [ ]:
# # Infer estimation of score
weights = get_weights_mc(prior, sample, n_sample_estim=10**5)
m_score = np.sum(score_sample * weights)

# For plottinf
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax2.set_ylabel("Density")
# ax1.set_ylabel("Score")


ax2.fill_between(xs, prior_norm.pdf(xs), color="0.7", alpha=0.2, label="prior")


loc_sample = sample.copy()
loc_weights = get_weights_mc(prior, loc_sample, n_sample_estim=10**6)

sorter = np.argsort(loc_sample.flatten())
loc_sample = loc_sample[sorter]
loc_weights = loc_weights[sorter]

cut = (loc_sample[1:] + loc_sample[:-1]) / 2

loc_weights_renorm = loc_weights.copy()
loc_weights_renorm[0] = 0.0
loc_weights_renorm[-1] = 0.0

loc_weights_renorm[1:-1] = loc_weights_renorm[1:-1] / (cut[1:, 0] - cut[:-1, 0])

cut_plot = [xs[0]] + list(np.array([[a, a] for a in cut]).flatten()) + [xs[-1]]
weights_for_plot = list(np.array([[a, a] for a in loc_weights_renorm]).flatten())
ax2.plot(cut_plot, weights_for_plot, color="black", label="Weights", linewidth=1)

ax2.plot(loc_sample, loc_weights_renorm, "o", color="tab:blue", markersize=4)
ax1.set_yticks([])
fig.tight_layout()

ax2.legend(loc=2)
plt.show()

Weights are computed for each parameter in the stack of evaluated parameter.

In [ ]:
T_s = gmap.T(sample)
T_approx = exp_approximation(Ts=T_s, scores=score_sample, weights=weights)

T_xs = gmap.T(xs[:, np.newaxis])
score_approx = (T_xs * T_approx).sum(-1)

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax2.set_ylabel("Density")
ax1.set_ylabel("Score")

ax2.fill_between(xs, prior_norm.pdf(xs), color="0.7", alpha=0.2, label="prior")
ax1.plot(
    xs, score(xs[:, np.newaxis]), "--", linewidth=1.0, color="black", label="Score"
)

ax1.plot(sample, score_sample, "o", markersize=9.0, label="Score eval.")
ax1.set_ylim(-0.3, 3.2)

score_sample_approx = (T_s * T_approx).sum(-1)

delta = np.sum(weights * (score_sample - score_sample_approx))

ax1.plot(xs, delta + score_approx, color="peru", linewidth=2.5, label="Score approx.")
fig.tight_layout()
ax1.legend(loc=1)
ax2.legend(loc=2)
plt.show()

The weighing process is used to compute the best L2 approximation of the score of a given form (here for gaussians, quadratic forms)

In [ ]:
delta = np.sum(weights * (score_sample - (T_s * T_approx).sum(-1)))

T_prior = gmap.param_to_T(prior_param)
T_updt_dir = -(temp**-1) * T_approx
T_new = T_prior + (1 - dampen) * T_updt_dir

post_param = gmap.T_to_param(T_new)

post = gmap(post_param)
post_norm = norm(loc=post.means[0], scale=np.sqrt(post.cov[0, 0]))

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax2.set_ylabel("Density")
ax1.set_ylabel("Score")

ax2.fill_between(xs, prior_norm.pdf(xs), color="0.7", alpha=0.2, label="prior")

ax1.set_ylim(-0.3, 3.2)

fig.tight_layout()

ax1.plot(xs, delta + score_approx, color="peru", linewidth=2.5, label="Score approx.")
fig.tight_layout()

ax2.fill_between(
    xs, post_norm.pdf(xs), color="skyblue", alpha=0.2, label="post. approx."
)
ax2.set_ylabel("Density")

ax1.set_ylabel("Score")

fig.tight_layout()
ax1.legend(loc=1)
ax2.legend(loc=2)
plt.show()

The approximated score is used to compute the posterior update.
This learning cycle is then repeated until either convergence is achieved or the maximal number of optimisation steps is achieved.

The algorithm is now showcased on a more complex function, two dimensional for plotting purposes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from surpbayes.bayes import pacbayes_minimize
from surpbayes.proba import GaussianMap, TensorizedGaussianMap

# For plotting purposes
from math import pi

angles = np.linspace(0, 2.001 * pi, 1000)
circle = np.array([np.cos(angles), np.sin(angles)])


def half_cov(cov):
    vals, vects = np.linalg.eigh(cov)
    return (np.sqrt(vals) * vects) @ vects.T


def repr_gauss(mean, cov, rad=1.0):
    """Visual representation of a 2D Gaussian distribution through ellipsis
    (representing highest density region, e.g. smallest volume region of a
    given probability (depending on rad))"""
    loc_circle = circle.copy()
    return mean + rad * (half_cov(cov) @ loc_circle).T


arr_1 = np.array([0.0, 1.0])
arr_2 = np.array([1, -1])
_shift = np.array([0.0, 0.5])


def score(x):
    z = x - _shift
    return 4 * np.arctan(0.5 * ((z @ arr_1 - 1.0) ** 2 + 100.0 * (z @ arr_2) ** 2))

In [ ]:
gmap = GaussianMap(2)

tic = time()
opt_res_new = pacbayes_minimize(
    score,
    gmap,
    temperature=0.1,  
    optimizer="score_approx",
    per_step=96,
    parallel=False,
    vectorized=True,
    print_rec=10,
    chain_length=141,
    n_estim_weights=2 * 10**4,
    kl_max=0.04,
    m_max=20,
    xtol=10**-7,
    kltol=10**-7,
    silent=False,
    dampen=0.1,
)
tac = time()
print("Time elapsed:", tac - tic)

for i, param in enumerate(opt_res_new.hist_param[:]):
    if i % 1 == 0:
        proba = gmap(param)
        proba_repr = repr_gauss(proba.means, proba.cov)
        plt.plot(proba_repr[:, 0], proba_repr[:, 1], color="black", linewidth=0.2)
        plt.plot(proba.means[0], proba.means[1], "x", color = "blue")

plt.title("Evolution of the posterior estimation")
plt.show()

We plot the evolutions of the posterior in conjonction with the score heatmap

In [ ]:
import seaborn as sns

x_min = -1.1
x_max = 1.8
n_x = 601

y_min = -1.1
y_max = 1.8
n_y = 601

x_axis_labels = np.linspace(
    x_min, x_max, n_x
)  # Avoid renormalisation issue at the angles
y_axis_labels = np.linspace(y_min, y_max, n_y)


def shift(xs, ys):
    return n_x * (xs - x_min) / (x_max - x_min), n_y * (ys - y_min) / (y_max - y_min)


values = np.array(np.meshgrid(y_axis_labels, x_axis_labels)).T

z = score(values)
alpha = np.linspace(0, 8, 100)

sns.heatmap(
    z.T,
    xticklabels=y_axis_labels,
    yticklabels=x_axis_labels,
    cmap=sns.color_palette("Blues", as_cmap=True),
)
# sns.heatmap(z)
for i, param in enumerate(opt_res_new.hist_param):
    if i % 2 == 0:
        proba = gmap(param)
        proba_repr = repr_gauss(proba.means, proba.cov)
        #         proba_repr  = repr_gauss(np.array([1.38,-1.1]), 0.01 * np.eye(2))

        xs, ys = shift(proba_repr[:, 0], proba_repr[:, 1])
        sns.lineplot(x=xs, y=ys, sort=False, color="yellow", linewidth=1.0)


proba = gmap(opt_res_new.opti_param)
proba_repr = repr_gauss(proba.means, proba.cov)

xs, ys = shift(proba_repr[:, 0], proba_repr[:, 1])
sns.lineplot(x=xs, y=ys, sort=False, color="red", linewidth=1.0)

plt.xticks([])
plt.yticks([])
# Note that the plot is turned compared to the previous version, due to seaborn's behavior for heatmap...


The procedure can also be investigated for other family of distributions, for instance Gaussians with fixed variance

In [ ]:
from surpbayes.proba import FactCovGaussianMap, FixedCovGaussianMap
from time import time

cov = np.array([[0.5, 0.4], [0.4, 0.5]])

ficgm = FixedCovGaussianMap(2, cov=cov) # Variance is fixed
facgm = FactCovGaussianMap(2, cov=cov) # Variance is fixed up to a multiplicative factor

proba_map = facgm
tic = time()
opt_res_new = pacbayes_minimize(
    score,
    proba_map,
    temperature=0.01,  
    optimizer="score_approx",
    per_step=160,
    parallel=False,
    vectorized=True,
    print_rec=10,
    chain_length=61,
    n_estim_weights=2*10**4,
    kl_max=0.05,
    m_max=20,
    xtol=10**-9,
    #     alpha_filter=0.99,
    silent=False,
    dampen=0.5,
)
tac = time()
print(tac - tic)
for i, param in enumerate(opt_res_new.hist_param):
    if i % 1 == 0:
        proba = proba_map(param)
        proba_repr = repr_gauss(proba.means, proba.cov)
        plt.plot(proba_repr[:, 0], proba_repr[:, 1], color="black", linewidth=0.2)
        plt.plot(proba.means[0], proba.means[1], "x")

plt.title("Evolution of the posterior estimation")
plt.show()

An interesting consideration is how much the posterior distribution gives weights to draws from previous distributions.

In [ ]:
from surpbayes.bayes.plot.surpac import plot_weights_per_gen

plot_weights_per_gen(opt_res_new, n_sample_estim_weight=10**6)

The evolution of the pushforward of the score by the posterior distribution can also be evaluated using the following plot (this is quite long to generate, since the weighing process is repeated for each previous parameter).

In [ ]:
from surpbayes.bayes.plot.optim_result import plot_score_evol

plot = plot_score_evol(opt_res_new, n_sample_weight_estim=10**4)
plot.show()

A simpler version of this plot can be obtained by using "plot_scores" function as follows:

In [ ]:
from surpbayes.bayes.plot import plot_hist_bayes, plot_scores

plot = plot_scores(sample_val=opt_res_new.sample_val, marker="x", s=0.03)
plot.show()

Finally, the evolution of the density of the pushforward of the score can be plotted using "plot_score_push_begin_end", which relies on a weighted kde estimator

In [ ]:
from surpbayes.bayes.plot import plot_score_push_begin_end

plot = plot_score_push_begin_end(opt_res_new)
plot.show()


## Gradient based algorithm

The gradient based routine can be called by setting the parameter 'pac_bayes_solver' to "corr_weights" or "knn". It is advised to use 'corr_weights'.
It is normal behavior that the optimisation procedure raises some ProbaBadGrad warnings. These indicate that a problematic gradient estimation was rejected as it damaged significantly the score. No need to worry about those.

In [ ]:
gauss_map = GaussianMap(2)

# We define the prior as the reference gaussian distribution, i.e. N(0,Id)
prior_param = gauss_map.ref_param

# To minimize PAC-Bayes bound, we use the pacbayes_minimize function.
opt_res_grad = pacbayes_minimize(
    score,
    gauss_map,
    prior_param=prior_param,
    temperature=0.1,  
    per_step=160,
    optimizer="corr_weights",
    gen_decay=np.log(1.2),
    k=160 * 20,
    parallel=False,
    vectorized=True,
    print_rec=50,
    chain_length=501,
    refuse_conf=0.9,
    momentum=0.9,
    eta=0.05,
    silent=False,
)

# It is normal behavior that the optimisation procedure raises some ProbaBadGrad warnings.
# These indicate that a problematic gradient estimation was rejected as it damaged significantly
# the score. No need to worry about those.

# We can access the parameter describing the posterior through the opti_param attribute
post_param = opt_res_grad.opti_param

In [ ]:
proba = gmap(opt_res_grad.hist_param[0])
proba_repr = repr_gauss(proba.means, proba.cov)
plt.plot(proba_repr[:, 0], proba_repr[:, 1], linewidth=1.0, label="prior")

for i, param in enumerate(opt_res_grad.hist_param[:]):
    if i % 3 == 0:
        proba = gmap(param)
        proba_repr = repr_gauss(proba.means, proba.cov)
        plt.plot(proba_repr[:, 0], proba_repr[:, 1], color="black", linewidth=0.2)
#         plt.plot(proba.means[0], proba.means[1], "x")

proba = gmap(opt_res_grad.opti_param)
proba_repr = repr_gauss(proba.means, proba.cov)
plt.plot(
    proba_repr[:, 0],
    proba_repr[:, 1],
    linewidth=1.2,
    color="crimson",
    label="posterior",
)
plt.plot([1], [1.5], "x", c="0.0", markersize=10, label="Glob. Min.")

plt.title("Evolution of the posterior estimation (corr weights)")
plt.legend()
plt.xticks([-1.0, 0.0, 1.0])
plt.yticks([-1.0, 0.0, 1.0])
plt.savefig("corr_weights_training.png")
plt.show()


# for i, param in enumerate(opt_res_grad.hist_param[:-450]):
#     if i % 2 == 0:
#         proba = gauss_map(param)
#         proba_repr = repr_gauss(proba.means, proba.cov)
#         plt.plot(proba_repr[:, 0], proba_repr[:, 1], color="black", linewidth=0.2)

In [ ]:
# The distribution then tries to shift towards the correct mean value
for i, param in enumerate(opt_res_grad.hist_param[50:500:30]):
    if i % 1 == 0:
        proba = gauss_map(param)
        proba_repr = repr_gauss(proba.means, proba.cov)
        plt.plot(proba_repr[:, 0], proba_repr[:, 1], color="black", linewidth=0.2)

In [ ]:
# The evolution of the PAC-Bayes objective can also be tracked:
plt.plot(opt_res_grad.hist_score)
plt.yscale("log")

In [ ]:
plot_score_push_begin_end(opt_res_grad)

## Uniform priors - Gaussian computations

The proba module offers a class of distributions on the hypercube benefitting from Gaussian like interpretation when the distribution are sufficiently concentrated and exact computations for KL.

In [ ]:
from surpbayes.proba.gauss import GaussHypercubeMap

dim = 2

# Toy score function
def score(x):
    return np.arctan(
        0.2 * (x @ np.array([1.0, 0.0], dtype=np.float64) - 0.6) ** 2
        + 20 * (x @ np.array([1.0, -2.0], dtype=np.float64)) ** 2
    )


pmap = GaussHypercubeMap(2)

Since this family of probability is of form $f(X)$ for $X$ spanning Gaussian, this defines an exponential family, and hence Catoni's bound can be minimized using SurPAC.

In [ ]:
opt_res = pacbayes_minimize(
    score,
    pmap,
    temperature=0.01,  
    per_step=80,
    dampen=0.1,
    kl_max=0.2,
    parallel=False,
    vectorized=True,
    print_rec=10,
    chain_length=51,
    silent=False,
)

The posterior can adapt to scores with strong identifiability issues such as Rosenbrock, since the probabilities can exhibit strong correlation structure

In [ ]:
import seaborn as sns

proba = pmap(opt_res.opti_param)
# The log density of the function can be accessed through log_dens
x_axis_labels = np.linspace(10**-4, 1 - 10**-4, 121)
y_axis_labels = np.linspace(10**-4, 1 - 10**-4, 121)

values = np.array(np.meshgrid(y_axis_labels, x_axis_labels)).T
z = proba.dens(values)

sns.heatmap(z, xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.title("Posterior distribution")
plt.xticks([])
plt.yticks([])

In [ ]:
plt.plot(opt_res.hist_score, label="Evolution of PAC-Bayes obj.")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
## Tests for problems of larger dimensions

d = 80
k = 5

blocks = [list(range(i * k, min(d, (i + 1) * k))) for i in range(int(np.ceil(d / k)))]

mat = (np.random.uniform(0.5, 2, d)) * np.random.normal(0, 1, (4 * d, d))
mat = mat.T @ mat / (4 * d)

print(np.linalg.eigvalsh(mat))

center = np.random.normal(0, 1, d)

In [ ]:
def score(xs):
    deltas = np.arctan(xs - center)
    return (deltas * (deltas @ mat)).sum(-1)


bgmap = BlockDiagGaussMap(blocks)

In [ ]:
opt_res = pacbayes_minimize(
    score,
    bgmap,
    per_step=800,
    kl_max=3.0,
    temperature=0.1,
    chain_length=20,
    vectorized=True,
)

In [ ]:
plt.plot(opt_res.hist_score)

# Checking stability

In [ ]:
def score(x):
    return -(
        1.2 * np.exp(-4 * (x @ [1] - 4) ** 2) + 1.0 * np.exp(-1.0 * (x @ [1]) ** 2)
    )


xs = np.linspace(-4, 8, 4000).reshape((4000, 1))

plt.plot(xs, score(xs))

In [ ]:
from surpbayes.accu_xy import AccuSampleVal

xs = np.linspace(3.5, 4.5, 1000).reshape((1000, 1))
accu = AccuSampleVal((1,), 1000)
accu.add(xs, score(xs))

In [ ]:
2 * (1 - norm.cdf(4))

In [ ]:
from scipy.stats import norm

gmap = GaussianMap(1)

tic = time()
opt_res_stable = pacbayes_minimize(
    score,
    gmap,
    #     facgm,
    #     prev_eval=accu, # Check if this improves matter or not
    temperature=0.002,  
    pac_bayes_solver="score_approx",
    prior_param=np.array([[0.0], [2.0]]),
    post_param=np.array([[4.0], [1.0]]),
    per_step=100,
    parallel=True,
    vectorized=False,
    print_rec=10,
    chain_length=101,
    n_estim_weights=10**6,
    kl_max=0.1,
    m_max=20,
    xtol=10**-8,
    kltol=10**-8,
    alpha_filter=1.0,
    silent=False,
    dampen=0.01,
)
tac = time()
print("Time elapsed:", tac - tic)

xs = np.linspace(-4, 6, 1000)

for i, param in enumerate(opt_res_stable.hist_param[:]):
    if i % 2 == 0:
        proba = gmap(param)
        plt.plot(
            xs,
            norm(proba.means[0], np.sqrt(proba.cov[0])).pdf(xs),
            color="black",
            linewidth=0.4,
        )
#         proba_repr = repr_gauss(proba.means, proba.cov)
#         plt.plot(proba_repr[:, 0], proba_repr[:, 1], color="black", linewidth=0.2)
#         plt.plot(proba.means[0], proba.means[1], "x")

plt.title("Evolution of the posterior estimation")
plt.show()

In [ ]:
opt_res_stable.opti_param

In [ ]:
for i, param in enumerate(opt_res_stable.hist_param[10:30]):
    if i % 1 == 0:
        proba = gmap(param)
        plt.plot(
            xs,
            norm(proba.means[0], np.sqrt(proba.cov[0])).pdf(xs),
            color="black",
            linewidth=0.4,
        )
#         proba_repr = repr_gauss(proba.means, proba.cov)
#         plt.plot(proba_repr[:, 0], proba_repr[:, 1], color="black", linewidth=0.2)
#         plt.plot(proba.means[0], proba.means[1], "x")

plt.title("Evolution of the posterior estimation")
plt.show()